#Algoritmo Apriori
---
```
Autor     : Bruno Moises Condori Ccarhuarupay
Fecha     : 20/11/2021
```
---

In [1]:
import pandas as pd
import numpy as np
import collections #manejo de los diccionarios y tuplas
from collections import defaultdict

#Implementacion

In [4]:
# Intersection devulve todos los conjuntos intersecados, estos son el soporte de determinado itemset.
def intersection_s(set_s,playlist_s):
    playlists_inter = []
    for song in set_s:
        playlists_inter.append(playlist_s[song])
    return len(set.intersection(*playlists_inter))
    
#Genera combinaciones en los itemsets    
def combinations(it, r):
    aux = tuple(it)
    n = len(aux)
    if r > n:
        return
    indices = list(range(r))
    yield tuple(aux[i] for i in indices)
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return
        indices[i] += 1
        for j in range(i + 1, r):
            indices[j] = indices[j - 1] + 1
        yield tuple(aux[i] for i in indices)

In [7]:
#Recibe la estructura de datos que contiene a las playlists y retorna una estructura con los itemsets fre-cuentes, bajo un umbral mínimo de soporte.
def get_frequent_itemsets(playlists, min_support):
  #Genera la frecuencia de cada cancion
  playlists = list(playlists.item().values())
  playlists = [set(playlist) for playlist in playlists]
  songes = [item for sublist in playlists for item in sublist]
  count_S = pd.Series(data=songes).value_counts().to_dict()
  #Se genera un diccionario de canciones y los indices 
  songs_pl = collections.defaultdict(set)
  for index, playlist in enumerate(playlists):
      for song in playlist:
          songs_pl[song].add(index)
  songs_pl = songs_pl
  count = {song: v for song, v in count_S.items() if v / len(playlists) >= min_support}
  itemset = [{song} for song in count.keys()]
  #Los itemsets mas frecuentes
  frecuency_itemsets = {}
  frecuency_itemsets[1] = sorted(count.items(), key=lambda x: x[1], reverse=True)
  frecuency_itemsets_size = []
  k = 2
  current = itemset
  while len(current) != 0:
    #Generar itemsets As
    combinations = set() 
    m = k - 2
    for A in current: 
      A = list(A)
      for aux_A in current:
        aux_A = list(aux_A)
        u = True
        for i in range(k - 2):
          if A[i] != aux_A[i]:
              u = False
              break
        if not u:
          continue
        if A[k - 2] < aux_A[k - 2]:
          c = A + [aux_A[k - 2]]
          c = frozenset(sorted(c))
          combinations.add(c)
    #verificacion del minimo de soporte para creacion de los itemsets.
    count_cb = {}   
    playlist_size = len(playlists)   
    for A in combinations:
        playlists_inter = []
        for song in A:
          playlists_inter.append(songs_pl[song])
        count_cb[A] = len(set.intersection(*playlists_inter))
    count2 = {subset: v for subset, v in count_cb.items() if v / playlist_size >= min_support} 
    dict2 = count2.keys() 
    frecuency_itemsets_size.extend(dict2)
    frecuency_itemsets[k] = sorted(count2.items(), key=lambda x: x[1], reverse=True)
    k += 1
    current = dict2

  sets = pd.DataFrame([item for sublist in frecuency_itemsets.values() for item in sublist]).round(3)
  sets.columns = ["itemset", "contador_support"]
  sets["support"] = sets["contador_support"] / len(playlists)

  output = []
  
  output.append(sets) # Itemsets mas frecuentes.
  output.append(playlists)# Datos de la playlist.
  output.append(songs_pl)# canciones de la playlists.
  output.append(frecuency_itemsets_size) # Itemsets.
  return output

In [3]:
#Recibe los itemsets frecuentes generados por la función anterior y retorna las reglas de asociación. 
#Se le puede entregar umbrales de confianza o lift para las reglas que se retornarán.
def generate_association_rules(frequent_itemsets, confidence, lift):
  rules = []
  for itemset in frequent_itemsets[3]:
    #Todas las combinaciones posibles de X -> Y.
    count = intersection_s(itemset,frequent_itemsets[2])
    for i in range(1, len(itemset) + 1):
        for x in combinations(itemset, i):
            x = set(x)
            y = set(itemset) - x
            x1 = intersection_s(x,frequent_itemsets[2]) / len(frequent_itemsets[0])
            xy1 = intersection_s(x.union(y),frequent_itemsets[2]) / len(frequent_itemsets[0])
            confidence_rule = xy1 / x1
            if len(x) > 0 and len(y) > 0:
                y1 = intersection_s(y,frequent_itemsets[2]) / len(frequent_itemsets[0])
                rule_lift = xy1 / (x1 * y1)
                rules.append((x, y, confidence_rule, xy1, rule_lift))
  rulesd1 = pd.DataFrame(data=rules, columns=["X", "Y", "confidence", "support", "lift"]).round(3)
  rulesd1["X"] = list(map(tuple, rulesd1["X"]))
  rulesd1["Y"] = list(map(tuple, rulesd1["Y"]))

  #Reglas de asociacion de 10 elementos del confidence
  order_by = "confidence"
  n = 10
  confidence_df = rulesd1.sort_values(order_by, ascending=False).head(n)

  #Reglas de asociacion de 10 elementos del lift
  order_by = "lift"
  n = 10
  lift_df2 = rulesd1.sort_values(order_by, ascending=False).head(n)

  #Hallar las reglas de asociacion que cumplan con la condicion de confianza y lift
  rulesd1["len_itemset"] = rulesd1.apply(lambda x: len(set(x["X"]).union(set(x["Y"]))), axis=1)
  rulesd1=rulesd1.sort_values(by="len_itemset", ascending=False)
  both_rules = rulesd1[(rulesd1["confidence"] >= confidence) & (rulesd1["lift"] >= lift)]

  return confidence_df,lift_df2,both_rules

Lectura de datos

In [5]:
db = np.load("spotify.npy", allow_pickle = True)

Creamos itemsets con un umbral de soporte bajo (0.01).

In [8]:
itemsets = get_frequent_itemsets(db,0.01)

Hallamos los 10 itemsets frecuentes con mayor soporte.

In [9]:
itemsets[0].sort_values(by="support").head()

,itemset,contador_support,support
1023,"(DNA., HUMBLE., Mask Off)",100,0.01
999,"(goosebumps, No Problem (feat. Lil Wayne & 2 C...",100,0.01
998,"(Fake Love, Mask Off)",100,0.01
997,"(No Problem (feat. Lil Wayne & 2 Chainz), Blac...",100,0.01
996,"(HUMBLE., Black Beatles)",100,0.01


Generar reglas de asociacion con una confianza de 0.5 y 1.2 de lift.

In [10]:
confidence_rules,lift_rules,both_rules = generate_association_rules(itemsets,0.5,1.2)

Imprimimos las 10 reglas con mayor confidence

In [11]:
confidence_rules

,X,Y,confidence,support,lift
404,"(DNA., Mask Off)","(HUMBLE.,)",0.909,0.098,2.002
386,"(DNA., XO TOUR Llif3)","(HUMBLE.,)",0.864,0.100,1.904
318,"(DNA.,)","(HUMBLE.,)",0.823,0.186,1.811
381,"(Mask Off, XO TOUR Llif3)","(HUMBLE.,)",0.804,0.128,1.770
373,"(Bounce Back, Broccoli (feat. Lil Yachty))","(Bad and Boujee (feat. Lil Uzi Vert),)",0.775,0.098,2.301
350,"(Slippery (feat. Gucci Mane), XO TOUR Llif3)","(HUMBLE.,)",0.765,0.099,1.685
345,"(Tunnel Vision, XO TOUR Llif3)","(HUMBLE.,)",0.750,0.100,1.652
363,"(Mask Off, Congratulations)","(HUMBLE.,)",0.747,0.118,1.645
416,"(goosebumps, Mask Off)","(HUMBLE.,)",0.743,0.107,1.637
428,"(Bounce Back, Mask Off)","(HUMBLE.,)",0.743,0.099,1.635


Imprimimos las 10 reglas ordenadas con mayor lift

In [12]:
lift_rules

,X,Y,confidence,support,lift
154,"(No Heart,)","(X (feat. Future),)",0.696,0.101,3.493
155,"(X (feat. Future),)","(No Heart,)",0.505,0.101,3.493
284,"(Chicken Fried,)","(Knee Deep (feat. Jimmy Buffett),)",0.500,0.104,3.261
285,"(Knee Deep (feat. Jimmy Buffett),)","(Chicken Fried,)",0.682,0.104,3.261
258,"(You Was Right,)","(Money Longer,)",0.603,0.114,3.183
259,"(Money Longer,)","(You Was Right,)",0.603,0.114,3.183
99,"(Butterfly Effect,)","(Bank Account,)",0.572,0.108,2.547
98,"(Bank Account,)","(Butterfly Effect,)",0.483,0.108,2.547
38,"(Bank Account,)","(Magnolia,)",0.491,0.110,2.541
39,"(Magnolia,)","(Bank Account,)",0.571,0.110,2.541


 Imprimimos las 10 reglas con la mayor confidence y lift.

In [13]:
both_rules.head(10)

,X,Y,confidence,support,lift,len_itemset
447,"(HUMBLE., Congratulations)","(XO TOUR Llif3,)",0.598,0.125,1.896,3
362,"(HUMBLE., Congratulations)","(Mask Off,)",0.565,0.118,1.832,3
367,"(iSpy (feat. Lil Yachty), HUMBLE.)","(Congratulations,)",0.703,0.106,1.787,3
368,"(iSpy (feat. Lil Yachty), Congratulations)","(HUMBLE.,)",0.673,0.106,1.482,3
369,"(HUMBLE., Congratulations)","(iSpy (feat. Lil Yachty),)",0.509,0.106,1.576,3
373,"(Bounce Back, Broccoli (feat. Lil Yachty))","(Bad and Boujee (feat. Lil Uzi Vert),)",0.775,0.098,2.301,3
374,"(Broccoli (feat. Lil Yachty), Bad and Boujee (...","(Bounce Back,)",0.645,0.098,2.224,3
375,"(Bounce Back, Bad and Boujee (feat. Lil Uzi Ve...","(Broccoli (feat. Lil Yachty),)",0.592,0.098,1.519,3
379,"(HUMBLE., Mask Off)","(XO TOUR Llif3,)",0.642,0.128,2.036,3
380,"(HUMBLE., XO TOUR Llif3)","(Mask Off,)",0.642,0.128,2.081,3


#Obtener 4 reglas

In [14]:
Aux_4=both_rules.head(4)
Aux_4

,X,Y,confidence,support,lift,len_itemset
447,"(HUMBLE., Congratulations)","(XO TOUR Llif3,)",0.598,0.125,1.896,3
362,"(HUMBLE., Congratulations)","(Mask Off,)",0.565,0.118,1.832,3
367,"(iSpy (feat. Lil Yachty), HUMBLE.)","(Congratulations,)",0.703,0.106,1.787,3
368,"(iSpy (feat. Lil Yachty), Congratulations)","(HUMBLE.,)",0.673,0.106,1.482,3


#Explicar reglas obtenidas

Para considerar sobre las 4 reglas obtenidas:
1.   Se puso un valor de umbral de soporte bajo (0.01) para que se pueda generar itemsets bastante discretos y faciles de analizar.
2.   El valor de lift indicara que la relacion de X e Y estan relacionados y de la misma forma con el support escogi ambos para que nos de los que cumplen con los estandares de mejor forma asi analizando podria decirce lo mejor de lo mejor.





**Regla 1**

```
(HUMBLE., Congratulations) -> (XO TOUR Llif3)
```
Ambas canciones son de genero hiphop/rap y su relacion estada por la misma el genero musical.
El valor de la confianza de la regla alcanza 0.598, es relativanmente baja considerando que ambos son del mismo genero musical, pero podemos alegar que este factor de confianza es bajo por los estilos que toman cada una.



**Regla 2**
```
(HUMBLE., Congratulations) -> (Mask Off)	
```
En esta regla pordemos observar que su valor de confianza es aun mas bajo 0.565, como en la regla anterior ambos tienen como genero HipHop, y aun asi su valor de confianza es muy bajo, en este caso se puede decir que tienen una confianza baja por la epoca y el estilo, ya que en la industria de la musica se puede decir que no es nada esta estatica, siempre se puede encontrar variantes del mismo genero musical.

**Regla 3**

```
(iSpy (feat. Lil Yachty), HUMBLE.) -> (Congratulations)	
```
Esta regla se nota que su factor de confianza es bueno ya que tiene 0.703, esto es debido a que estas canciones son del mismo artista.

**Regla 4**

```
(iSpy (feat. Lil Yachty), Congratulations)	(HUMBLE.,)	
```
Esta cancion vuelve a aparecer al igual que la anterior regla por lo que se tiene una fuerte relacion ya que en esta oportunidad tiene un factor de confianza de 0.673, de esta regla podemos decir si escuchas a un artista te gustara toda su discografia.